In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 3.8 MB 11.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.7 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 895 kB 52.0 MB/s 
     |████████████████████████████████| 6.5 MB 42.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=b8d80e2b919df6db6ff7da69dcf51f0cd297a036aff3081f99ff84e1279f6db9
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive 
drive.mount('/content/drive') 
%cd /content/drive/My\ Drive/Colab Notebooks/Q&A Bot/Scholarship Q&A

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Q&A Bot/Scholarship Q&A


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import os
import numpy as np
import re

In [ ]:
count = 0
dir_path = '/content/drive/MyDrive/Colab Notebooks/Q&A Bot/Scholarship Q&A/answers'
for path in os.scandir(dir_path):
    if path.is_file():
        count += 1
print('file count:', count)

file count: 9


In [ ]:
df_array = []
for i in range(1,count+1):
    df_array.append(pd.read_csv("answers/data"+str(i)+".csv"))

raw_data_robert = pd.concat(df_array)

In [ ]:
raw_data_bert = pd.read_csv('raw_data_bert.csv')
raw_data_bert.insert(3, 'score2', raw_data_robert['score'].values)

In [ ]:
raw_data_bert.drop( raw_data_bert[ (raw_data_bert['score'] * raw_data_bert['score2'] < 0.01) | (raw_data_bert['score']< 0.075) | (raw_data_bert['score2']< 0.075)].index , inplace=True)
raw_data_bert.index = pd.RangeIndex(len(raw_data_bert.index))

In [ ]:
raw_data_bert.to_csv("model_score_filtered.csv", index=False)

In [ ]:
#for df in df_array:
#    df.drop(df[df['score'] < 0.01].index, inplace = True)
#    df.index = pd.RangeIndex(len(df.index))

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')


In [ ]:
# full data

similarities_answer = []
similarities_topic = []
multiplication = []

questions = raw_data_bert.question.unique()

topics = ["score","score","institution","grade","place","money","ethnicity","race","nationality","age","gender","religion","ability","disability","major","date","score","institution"]

for question,topic in zip(questions,topics):

    sentences = raw_data_bert[raw_data_bert.question == question].answer.to_list()
    scores = raw_data_bert[raw_data_bert.question == question].score.to_list()
    scores2 = raw_data_bert[raw_data_bert.question == question].score2.to_list()

    
    sentence_embeddings = model.encode(sentences)
    
    topic_embedding = model.encode(topic)
    
    for sentence_embedding, score, score2 in zip(sentence_embeddings, scores, scores2):

        similarity = cosine_similarity([sentence_embedding],sentence_embeddings[:])
        sum = similarity.sum() - 1 # Substract itself "1"
        sim_ans = sum/len(sentences)
        similarities_answer.append(sim_ans) 

        sim_topic = cosine_similarity([sentence_embedding],[topic_embedding])[0][0]
        if(sim_topic<0):
            similarities_topic.append(0)
        else:
            similarities_topic.append(sim_topic)
        
        mul = score*score2*sim_ans*sim_topic
        if mul < 0:
          multiplication.append(0)
        else:
          multiplication.append(mul)

raw_data_bert.insert(4, 'sim_ans_ans_score', similarities_answer)
raw_data_bert.insert(5, 'sim_topic_ans_score', similarities_topic)
raw_data_bert.insert(6, 'multiplication', multiplication)

In [ ]:
raw_data_bert.to_csv("data_with_similarity_scores.csv", index=False)

In [ ]:
# similarity

questions = raw_data_bert.question.unique()

for question in questions:

    sliced_data = raw_data_bert[raw_data_bert.question == question]
    
    mean = sliced_data.sim_ans_ans_score.mean()
    std = sliced_data.sim_ans_ans_score.std()

    normalized_dist = (sliced_data.sim_ans_ans_score.values - mean)/std

    indexes = np.where(normalized_dist < normalized_dist.mean()-3*normalized_dist.std())[0]
    false_slices = sliced_data.iloc[indexes].index

    raw_data_bert.drop(raw_data_bert[raw_data_bert.index.isin(false_slices)].index, inplace = True)
    
raw_data_bert.index = pd.RangeIndex(len(raw_data_bert.index))

In [ ]:
raw_data_bert.to_csv("filtered_similarity.csv", index=False)

In [ ]:
raw_data_bert = pd.read_csv("filtered_similarity.csv")

In [ ]:
all_mean = raw_data_bert.sim_ans_ans_score.mean()*raw_data_bert.sim_topic_ans_score.mean()*raw_data_bert.score.mean()*raw_data_bert.score2.mean()
all_std = raw_data_bert.sim_ans_ans_score.std()*raw_data_bert.sim_topic_ans_score.std()*raw_data_bert.score.std()*raw_data_bert.score2.std()

In [ ]:
raw_data_bert[raw_data_bert.multiplication > all_mean - 3*all_std].to_csv('final_data.csv', index=False)